In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import csv
import requests
from zipfile import ZipFile
import os
from datetime import date

In [2]:
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile

In [3]:
yearDate = date.today().strftime('%m%d%y')
print(yearDate)

111320


In [4]:
daily_covid_county_service_id = 'a7887f1940b34bf5a02c6f7f27a5cb2c'
gis = GIS()
data_cases = gis.content.get(daily_covid_county_service_id)
data_cases

<Item title:"Florida COVID19 Cases by County" type:Feature Layer Collection owner:Epidemiology.GIS>

In [5]:
query = data_cases.layers[0].query(where="1=1")
df_covid_cases = query.sdf
df_covid_cases.head()

,OBJECTID,OBJECTID_12_13,DEPCODE,COUNTY,COUNTYNAME,County_1,State,PUIsTotal,Age_0_4,Age_5_14,...,MonNow,Shape__Area,Shape__Length,GlobalID,NewPos,NewNeg,NewTested,NewPercPos,MedianAge,SHAPE
0,1,1,21,041,GILCHRIST,Gilchrist,FL,1474,38,84,...,722,9.204908e+08,148547.348006,cb6285a2-da2e-4583-aad5-1221776836ae,0,75,75,0.000000,41,"{""rings"": [[[529493.625602742, 647744.81243718..."
1,2,2,54,107,PUTNAM,Putnam,FL,6277,174,563,...,1595,2.142439e+09,232574.925675,ce511c5d-2638-44a3-b2f8-b13847543637,6,206,212,0.028302,40,"{""rings"": [[[633424.87518893, 650359.936015058..."
2,3,3,62,123,TAYLOR,Taylor,FL,2806,65,166,...,1552,2.685253e+09,267348.178950,541bd5a3-4453-499a-9896-f0edc5353d0b,2,128,130,0.015385,40,"{""rings"": [[[425905.96838189, 699605.625594474..."
3,4,4,46,091,OKALOOSA,Okaloosa,FL,15864,329,1122,...,10439,2.369223e+09,330553.160278,eb5d2450-a892-40fd-9ba4-9821a7438525,110,562,672,0.163690,40,"{""rings"": [[[170978.169305377, 740209.62491612..."
4,5,5,7,013,CALHOUN,Calhoun,FL,1560,33,120,...,1118,1.487627e+09,192207.890331,3b55aa95-5170-4caf-82a3-e99ad7459349,10,89,99,0.101010,44,"{""rings"": [[[310646.592981238, 733517.62524545..."


In [6]:
df_covid_cases_1 = df_covid_cases.drop(columns=['SHAPE'])
df_Case = df_covid_cases_1.sort_values(by=['DEPCODE'])
df_Case.head()

,OBJECTID,OBJECTID_12_13,DEPCODE,COUNTY,COUNTYNAME,County_1,State,PUIsTotal,Age_0_4,Age_5_14,...,EverMon,MonNow,Shape__Area,Shape__Length,GlobalID,NewPos,NewNeg,NewTested,NewPercPos,MedianAge
65,66,66,1,001,ALACHUA,Alachua,FL,26524,414,681,...,1752,1729,2.510756e+09,272135.471452,9eddb8fb-00dd-45f1-a09d-8e689049e28a,75,1083,1158,0.064767,24
16,17,17,2,003,BAKER,Baker,FL,3146,71,153,...,2209,2208,1.520246e+09,183737.354704,393e3bc9-079e-4ad6-9162-d77ee72feb59,9,51,60,0.150000,41
14,15,15,3,005,BAY,Bay,FL,10956,138,328,...,1985,1978,1.930771e+09,558412.990826,a66f3b06-8ee0-4dec-843e-f41d712aad2f,51,412,463,0.110151,42
39,40,40,4,007,BRADFORD,Bradford,FL,2966,69,160,...,1896,1865,7.767446e+08,167178.838253,588f1d06-056b-4af5-a2e7-a5366c833ba5,2,69,71,0.028169,46
41,42,42,5,009,BREVARD,Brevard,FL,32305,415,992,...,3245,3187,2.687644e+09,793218.333609,943db4a8-472c-444e-b81a-0e7cf9758ca2,145,1312,1457,0.099520,44


In [7]:
cases_by_population_id = '09e0d01d8e8e46b6ae678c256d918178'
gis = GIS()
data_popu = gis.content.get(cases_by_population_id)
data_popu

<Item title:"Case Maps for COVID19" type:Feature Layer Collection owner:Epidemiology.GIS>

In [8]:
query_popu = data_popu.layers[0].query(where="1=1")
df_cases_popu = query_popu.sdf
df_cases_popu.head()

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,CovidRate,Shape__Area,Shape__Length,SHAPE
0,1,Alachua,12001,263148,0,75,556,1063,11523,1,4379,2.509300e+09,262028.034556,"{""rings"": [[[529453.427928006, 647729.16620529..."
1,2,Baker,12003,27785,0,9,44,94,1880,2,6766,1.524801e+09,177464.398185,"{""rings"": [[[547611.939311098, 731650.13314648..."
2,3,Bay,12005,182482,0,51,339,548,7198,3,3944,2.285559e+09,233326.432744,"{""rings"": [[[208213.290884909, 697920.20090373..."
3,4,Bradford,12007,26979,0,2,36,55,1293,4,4793,7.782856e+08,161130.117599,"{""rings"": [[[552463.851341891, 658321.23030116..."
4,5,Brevard,12009,576808,0,145,872,1653,13233,5,2294,3.353781e+09,317459.583010,"{""rings"": [[[694017.58758627, 515464.701569436..."


In [9]:
df_cases_popu_1 = df_cases_popu.drop(columns=['SHAPE'])
df_PP = df_cases_popu_1.sort_values(by=['FID'])
df_PP.head()

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,CovidRate,Shape__Area,Shape__Length
0,1,Alachua,12001,263148,0,75,556,1063,11523,1,4379,2.509300e+09,262028.034556
1,2,Baker,12003,27785,0,9,44,94,1880,2,6766,1.524801e+09,177464.398185
2,3,Bay,12005,182482,0,51,339,548,7198,3,3944,2.285559e+09,233326.432744
3,4,Bradford,12007,26979,0,2,36,55,1293,4,4793,7.782856e+08,161130.117599
4,5,Brevard,12009,576808,0,145,872,1653,13233,5,2294,3.353781e+09,317459.583010


In [10]:
with pd.ExcelWriter('data/Florida_COVID19_Cases_by_County{}.xlsx'.format(yearDate)) as writer:  
    df_Case.to_excel(writer, sheet_name='Florida_COVID19_Cases_by_County')
    df_PP.to_excel(writer, sheet_name='Sheet1')

In [11]:
df_case = df_Case[['OBJECTID','COUNTYNAME','County_1','State','NewPos','NewTested','PUIsTotal','CasesAll']].copy()
df_case.head()

,OBJECTID,COUNTYNAME,County_1,State,NewPos,NewTested,PUIsTotal,CasesAll
65,66,ALACHUA,Alachua,FL,75,1158,26524,11631
16,17,BAKER,Baker,FL,9,60,3146,1888
14,15,BAY,Bay,FL,51,463,10956,7440
39,40,BRADFORD,Bradford,FL,2,71,2966,1295
41,42,BREVARD,Brevard,FL,145,1457,32305,13312


In [12]:
a = set(df_PP['NAME'].values)
b = set(df_case['County_1'].values)
a.difference(b)

{'A State'}

In [13]:
b.difference(a)

{'State'}

In [14]:
df_case.loc[df_case['County_1']=='State','County_1']='A State'

In [15]:
df_merge = pd.merge(df_PP, df_case, left_on=['NAME'],right_on= ['County_1'], how = 'outer')
df_merge.head(len(df_merge))

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,...,Shape__Area,Shape__Length,OBJECTID,COUNTYNAME,County_1,State,NewPos,NewTested,PUIsTotal,CasesAll
0,1,Alachua,12001,263148,0,75,556,1063,11523,1,...,2.509300e+09,262028.034556,66,ALACHUA,Alachua,FL,75,1158,26524,11631
1,2,Baker,12003,27785,0,9,44,94,1880,2,...,1.524801e+09,177464.398185,17,BAKER,Baker,FL,9,60,3146,1888
2,3,Bay,12005,182482,0,51,339,548,7198,3,...,2.285559e+09,233326.432744,15,BAY,Bay,FL,51,463,10956,7440
3,4,Bradford,12007,26979,0,2,36,55,1293,4,...,7.782856e+08,161130.117599,40,BRADFORD,Bradford,FL,2,71,2966,1295
4,5,Brevard,12009,576808,0,145,872,1653,13233,5,...,3.353781e+09,317459.583010,42,BREVARD,Brevard,FL,145,1457,32305,13312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,12129,31877,0,15,77,123,1437,65,...,1.633086e+09,234986.413040,41,WAKULLA,Wakulla,FL,15,158,3374,1446
65,66,Walton,12131,65858,0,39,432,650,2881,66,...,2.973057e+09,274860.715452,51,WALTON,Walton,FL,39,163,5203,3183
66,67,Washington,12133,24566,0,22,55,99,1331,67,...,1.595344e+09,237797.047191,53,WASHINGTON,Washington,FL,22,52,2381,1343
67,68,Unknown,99999,0,0,1,6,9,151,68,...,NaN,NaN,69,Unknown,Unknown,FL,1,486,3177,2056


In [16]:
df_merge_extract = df_merge.drop(columns=['OBJECTID','FID_1', 'COUNTYNAME'])
df_merge_extract.head()

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,Shape__Area,Shape__Length,County_1,State,NewPos,NewTested,PUIsTotal,CasesAll
0,1,Alachua,12001,263148,0,75,556,1063,11523,4379,2.509300e+09,262028.034556,Alachua,FL,75,1158,26524,11631
1,2,Baker,12003,27785,0,9,44,94,1880,6766,1.524801e+09,177464.398185,Baker,FL,9,60,3146,1888
2,3,Bay,12005,182482,0,51,339,548,7198,3944,2.285559e+09,233326.432744,Bay,FL,51,463,10956,7440
3,4,Bradford,12007,26979,0,2,36,55,1293,4793,7.782856e+08,161130.117599,Bradford,FL,2,71,2966,1295
4,5,Brevard,12009,576808,0,145,872,1653,13233,2294,3.353781e+09,317459.583010,Brevard,FL,145,1457,32305,13312


In [17]:
df_merge_extract['P100K_Y'] = df_merge_extract['NewPos'] / df_merge_extract['Pop']*100000
df_merge_extract['P100K_7'] = df_merge_extract['Css7Dys'] / df_merge_extract['Pop']*100000
df_merge_extract['P100K_14'] = df_merge_extract['Css14Dy'] / df_merge_extract['Pop']*100000
df_merge_extract['T100K_Y'] = df_merge_extract['NewTested'] / df_merge_extract['Pop']*100000
df_merge_extract['PRate'] = df_merge_extract['NewPos'] / df_merge_extract['NewTested']
df_merge_extract['TestGoal'] = df_merge_extract['Css14Dy'] / 14 * 30

In [18]:
df_merge_drop_unknown = df_merge_extract.drop([67])
df_merge_drop_unknown

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,NewPos,NewTested,PUIsTotal,CasesAll,P100K_Y,P100K_7,P100K_14,T100K_Y,PRate,TestGoal
0,1,Alachua,12001,263148,0,75,556,1063,11523,4379,...,75,1158,26524,11631,28.501072,211.287944,403.955189,440.056546,0.064767,2277.857143
1,2,Baker,12003,27785,0,9,44,94,1880,6766,...,9,60,3146,1888,32.391578,158.358827,338.312039,215.943855,0.150000,201.428571
2,3,Bay,12005,182482,0,51,339,548,7198,3944,...,51,463,10956,7440,27.947962,185.771747,300.303592,253.723655,0.110151,1174.285714
3,4,Bradford,12007,26979,0,2,36,55,1293,4793,...,2,71,2966,1295,7.413173,133.437118,203.862263,263.167649,0.028169,117.857143
4,5,Brevard,12009,576808,0,145,872,1653,13233,2294,...,145,1457,32305,13312,25.138348,151.176821,286.577163,252.597051,0.099520,3542.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,12127,527634,0,129,675,1269,14008,2655,...,129,1637,35043,14117,24.448766,127.929588,240.507625,310.252940,0.078803,2719.285714
64,65,Wakulla,12129,31877,0,15,77,123,1437,4508,...,15,158,3374,1446,47.055871,241.553471,385.858142,495.655175,0.094937,263.571429
65,66,Walton,12131,65858,0,39,432,650,2881,4375,...,39,163,5203,3183,59.218318,655.956755,986.971970,247.502202,0.239264,1392.857143
66,67,Washington,12133,24566,0,22,55,99,1331,5418,...,22,52,2381,1343,89.554669,223.886673,402.996011,211.674672,0.423077,212.142857


In [19]:
df_merge_drop_unknown['P100K_Y'] =df_merge_drop_unknown['P100K_Y'].astype(int)
df_merge_drop_unknown['P100K_7'] =df_merge_drop_unknown['P100K_7'].astype(int)
df_merge_drop_unknown['P100K_14'] =df_merge_drop_unknown['P100K_14'].astype(int)
df_merge_drop_unknown['T100K_Y'] =df_merge_drop_unknown['T100K_Y'].astype(int)

In [20]:
df_merge_drop_unknown.head(len(df_merge_drop_unknown))

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,NewPos,NewTested,PUIsTotal,CasesAll,P100K_Y,P100K_7,P100K_14,T100K_Y,PRate,TestGoal
0,1,Alachua,12001,263148,0,75,556,1063,11523,4379,...,75,1158,26524,11631,28,211,403,440,0.064767,2277.857143
1,2,Baker,12003,27785,0,9,44,94,1880,6766,...,9,60,3146,1888,32,158,338,215,0.150000,201.428571
2,3,Bay,12005,182482,0,51,339,548,7198,3944,...,51,463,10956,7440,27,185,300,253,0.110151,1174.285714
3,4,Bradford,12007,26979,0,2,36,55,1293,4793,...,2,71,2966,1295,7,133,203,263,0.028169,117.857143
4,5,Brevard,12009,576808,0,145,872,1653,13233,2294,...,145,1457,32305,13312,25,151,286,252,0.099520,3542.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,12127,527634,0,129,675,1269,14008,2655,...,129,1637,35043,14117,24,127,240,310,0.078803,2719.285714
64,65,Wakulla,12129,31877,0,15,77,123,1437,4508,...,15,158,3374,1446,47,241,385,495,0.094937,263.571429
65,66,Walton,12131,65858,0,39,432,650,2881,4375,...,39,163,5203,3183,59,655,986,247,0.239264,1392.857143
66,67,Washington,12133,24566,0,22,55,99,1331,5418,...,22,52,2381,1343,89,223,402,211,0.423077,212.142857


In [21]:
gdf_county_polygon = gpd.read_file('shp/Florida_Counties.shp')
gdf_county_polygon.head()

,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,21,041,Gilchrist,2000-05-16,9.908353e+09,4.873000e+05,"POLYGON ((-82.65814 29.83028, -82.65803 29.822..."
1,2,54,107,Putnam,2000-05-16,2.305869e+10,7.629677e+05,"POLYGON ((-81.58084 29.83956, -81.57893 29.835..."
2,3,62,123,Taylor,2000-05-16,2.891747e+10,8.772527e+05,"MULTIPOLYGON (((-83.73037 30.30358, -83.70368 ..."
3,4,46,091,Okaloosa,2000-05-16,2.562159e+10,1.087058e+06,"MULTIPOLYGON (((-86.39159 30.64970, -86.39175 ..."
4,5,7,013,Calhoun,2000-05-16,1.604809e+10,6.313440e+05,"POLYGON ((-84.93266 30.60637, -84.93287 30.605..."


In [22]:
a = set(df_merge_drop_unknown['County_1'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'A State'}

In [23]:
b.difference(a)

set()

In [24]:
df_merge_poly = pd.merge(df_merge_drop_unknown, gdf_county_polygon, left_on = ['County_1'],right_on= ['COUNTYNAME'], how = 'outer')
df_merge_poly.head(len(df_merge_poly))

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,PRate,TestGoal,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,Alachua,12001,263148,0,75,556,1063,11523,4379,...,0.064767,2277.857143,66.0,1.0,001,Alachua,2000-05-16,2.702324e+10,8.927266e+05,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,2,Baker,12003,27785,0,9,44,94,1880,6766,...,0.150000,201.428571,17.0,2.0,003,Baker,2000-05-16,1.636216e+10,6.027173e+05,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,3,Bay,12005,182482,0,51,339,548,7198,3944,...,0.110151,1174.285714,15.0,3.0,005,Bay,2000-05-16,2.084232e+10,1.834701e+06,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,4,Bradford,12007,26979,0,2,36,55,1293,4793,...,0.028169,117.857143,40.0,4.0,007,Bradford,2000-05-16,8.359863e+09,5.483267e+05,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,5,Brevard,12009,576808,0,145,872,1653,13233,2294,...,0.099520,3542.142857,42.0,5.0,009,Brevard,2000-05-16,2.893695e+10,2.600567e+06,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,12127,527634,0,129,675,1269,14008,2655,...,0.078803,2719.285714,25.0,64.0,127,Volusia,2000-05-16,3.364565e+10,1.837936e+06,"MULTIPOLYGON (((-81.09705 29.35180, -81.09292 ..."
64,65,Wakulla,12129,31877,0,15,77,123,1437,4508,...,0.094937,263.571429,41.0,65.0,129,Wakulla,2000-05-16,1.702251e+10,9.167339e+05,"MULTIPOLYGON (((-84.41197 30.30244, -84.37858 ..."
65,66,Walton,12131,65858,0,39,432,650,2881,4375,...,0.239264,1392.857143,51.0,66.0,131,Walton,2000-05-16,2.926099e+10,1.216747e+06,"POLYGON ((-86.03523 30.98825, -86.03532 30.982..."
66,67,Washington,12133,24566,0,22,55,99,1331,5418,...,0.423077,212.142857,53.0,67.0,133,Washington,2000-05-16,1.721229e+10,8.010119e+05,"POLYGON ((-85.43543 30.56727, -85.46843 30.567..."


In [25]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [26]:
shpfile_name = "covid_impact_poly"+ yearDate
gdf_merge_poly = gpd.GeoDataFrame(df_merge_poly, geometry='geometry',crs="EPSG:4326")
gdf_merge_poly.to_file(os.path.join(result_folder,"{}.shp".format(shpfile_name)))

In [27]:
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))   

covid_impact_poly111320.prj
covid_impact_poly111320.shp
covid_impact_poly111320.cpg
covid_impact_poly111320.shx
covid_impact_poly111320.dbf


In [28]:
print('Done!!!!')

Done!!!!
